# Architecture du Chatbot RAG pour la Constitution française

## Vue d'ensemble

Nous implémentons un chatbot basé sur la technique de Retrieval Augmented Generation (RAG) pour répondre aux questions sur la Constitution française. L'architecture se compose de plusieurs modules interconnectés, chacun responsable d'une partie spécifique du processus.

## Composants principaux

1. **PDF Processor (`pdf_processor.py`)**
   - Responsable du chargement et de la segmentation du PDF de la Constitution.
   - Utilise `PyPDFLoader` de Langchain pour charger le PDF.
   - Implémente une logique personnalisée pour diviser le texte en chunks structurés (préambule, titres, articles).

2. **Embedding Generator (`embedding_generator.py`)**
   - Génère des embeddings pour les chunks de texte.
   - Utilise le modèle `BAAI/bge-large-en-v1.5` de SentenceTransformer pour la génération d'embeddings.
   - Stocke les embeddings dans une base de données vectorielle ChromaDB.

3. **Semantic Search (`semantic_search.py`)**
   - Effectue des recherches sémantiques dans la base de données d'embeddings.
   - Utilise ChromaDB pour la recherche rapide des chunks les plus pertinents.

4. **LLM Interface (`llm_interface.py`)**
   - Fournit une interface pour interagir avec les modèles de langage (LLM).
   - Utilise une API locale (probablement Ollama) pour accéder aux modèles Mistral et Phi-2.

5. **Main Script (`main.py`)**
   - Sert d'interface utilisateur et d'orchestrateur pour le chatbot.
   - Gère le flux de travail complet, de la réception de la question à la génération de la réponse.

## Flux de travail

1. **Prétraitement (effectué une seule fois)**
   - Le PDF est chargé et divisé en chunks structurés.
   - Les embeddings sont générés pour chaque chunk et stockés dans ChromaDB.

2. **Processus de réponse aux questions**
   - L'utilisateur pose une question via l'interface console.
   - La question est utilisée pour effectuer une recherche sémantique dans ChromaDB.
   - Les chunks les plus pertinents sont récupérés.
   - Un prompt est construit en combinant la question et les chunks pertinents.
   - Le prompt est envoyé au LLM via l'interface LLM.
   - La réponse générée par le LLM est affichée à l'utilisateur, avec les sources utilisées.

## Caractéristiques clés

- **Segmentation intelligente** : Le PDF est divisé de manière à préserver la structure de la Constitution (préambule, titres, articles).
- **Recherche sémantique avancée** : Utilisation d'un modèle d'embedding performant pour une recherche précise.
- **Flexibilité du LLM** : Possibilité de choisir entre différents modèles (Mistral, Phi-2).
- **Contextualisation** : La réponse est générée en tenant compte du contexte spécifique de la Constitution.
- **Traçabilité** : Les sources utilisées pour générer la réponse sont fournies à l'utilisateur.

## 1. Chargement et prétraitement du pdf : `pdf_processor.py`

### Classe `PDFProcessor`
- **Initialisation** : Utilise `PyPDFLoader` de Langchain pour charger le PDF.
- **Méthode `load_and_split()`** :
  - Charge le PDF et extrait le texte complet.
  - Appelle `split_by_structure()` pour segmenter le texte.
- **Méthode `split_by_structure(text)`** :
  - Utilise des expressions régulières pour diviser le texte en chunks structurés.
  - Extrait le préambule (avant l'Article 1).
  - Divise le reste en titres et articles.
  - Crée des objets `Document` avec métadonnées (titre, numéro d'article).
- **Méthode `process()`** :
  - Point d'entrée principal qui appelle `load_and_split()`.
  - Affiche un aperçu des chunks pour vérification.

**Techniques clés** :
- Utilisation d'expressions régulières pour une segmentation précise.
- Conservation de la structure du document (titres, articles) dans les métadonnées.

In [1]:
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document

class PDFProcessor:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.loader = PyPDFLoader(pdf_path)

    def load_and_split(self):
        pages = self.loader.load_and_split()
        full_text = ' '.join([page.page_content for page in pages])
        chunks = self.split_by_structure(full_text)
        return chunks

    def split_by_structure(self, text):
        chunks = []
        
        # Extraire le contenu avant l'Article 1 (Chunk 0)
        pre_article_1 = re.split(r'ARTICLE\s+PREMIER\.?', text)[0]
        chunks.append(Document(
            page_content=pre_article_1.strip(),
            metadata={
                "source": self.pdf_path,
                "title": "Préambule",
                "article_num": "0"
            }
        ))

        # Diviser le reste du texte en titres et articles
        main_text = text[len(pre_article_1):]
        title_pattern = r'Titre\s+[IVX]+\s*\n([^\n]+)'
        article_pattern = r'ARTICLE\s+(\w+(?:\-\d+)?)\.?([^A]+)(?=ARTICLE|$)'
        
        current_title = ""
        for title_match in re.finditer(title_pattern, main_text):
            title = title_match.group(0).strip()
            current_title = title
            title_end = title_match.end()
            next_title_match = re.search(title_pattern, main_text[title_end:])
            title_content = main_text[title_end:next_title_match.start() + title_end] if next_title_match else main_text[title_end:]
            
            for article_match in re.finditer(article_pattern, title_content):
                article_num = article_match.group(1)
                article_content = article_match.group(2).strip()
                
                chunk = Document(
                    page_content=f"ARTICLE {article_num}.\n{article_content}",
                    metadata={
                        "source": self.pdf_path,
                        "title": current_title,
                        "article_num": article_num
                    }
                )
                chunks.append(chunk)

        return chunks

    def process(self):
        chunks = self.load_and_split()
        print(f"Le document a été découpé en {len(chunks)} chunks.")
        
        # Afficher un aperçu des chunks pour vérification
        for i, chunk in enumerate(chunks[:4]):  # Afficher les 10 premiers chunks
            print(f"\nChunk {i}:")
            print(f"Titre: {chunk.metadata['title']}")
            print(f"Article: {chunk.metadata['article_num']}")
            print(chunk.page_content[:200] + "...")  # Afficher les 200 premiers caractères
        
        return chunks

# Exemple d'utilisation
if __name__ == "__main__":
    pdf_path = "constitution.pdf"
    processor = PDFProcessor(pdf_path)
    chunks = processor.process()

Le document a été découpé en 79 chunks.

Chunk 0:
Titre: Préambule
Article: 0
1 janvier 2015  
 
 
CONSTITUTION  
 
 
 
Le Gouvernement de la République, conformément 
à la loi constitutionnelle du 3  juin 1958, a proposé,  
 Le peuple français a adopté,  
 
Le Président de la ...

Chunk 1:
Titre: Titre II 
LE PRÉSIDENT DE LA R ÉPUBLIQUE
Article: 5
ARTICLE 5.
Le Président de la République veille au respect de la Co nstitution. Il 
assure, par son arbitrage, le fonctionnement régulier des pouvoirs publics ainsi que 
la continuité de l'État.  
Il ...

Chunk 2:
Titre: Titre II 
LE PRÉSIDENT DE LA R ÉPUBLIQUE
Article: 6
ARTICLE 6.
Le Président de la République est élu pour cinq ans au suffrage 
universel direct.  
Nul ne peut exercer plus de deux mandats consécutifs.  
Les modalités d'application du présent article s...

Chunk 3:
Titre: Titre II 
LE PRÉSIDENT DE LA R ÉPUBLIQUE
Article: 7
ARTICLE 7.
Le Président de la République est é lu à la majorité absolue des 
suffrages exprimés. Si cel

## 2. Extraction et stockage de embeddings : `embedding_generator.py`

### Classe `EmbeddingGenerator`
- **Initialisation** : 
  - Charge le modèle SentenceTransformer 'BAAI/bge-large-en-v1.5'.
  - Configure ChromaDB pour le stockage persistant.
- **Méthode `generate_embeddings(chunks)`** :
  - Génère des embeddings pour les chunks de texte.
  - Applique une normalisation L2 aux embeddings.
- **Méthode `store_embeddings(chunks, embeddings)`** :
  - Stocke les embeddings, textes et métadonnées dans ChromaDB.
- **Méthode `process_and_store(chunks)`** :
  - Combine la génération et le stockage des embeddings.
- **Méthode `get_similar_chunks(query, n_results)`** :
  - Effectue une recherche sémantique dans ChromaDB.
- **Méthode `update_embeddings(chunks)`** :
  - Met à jour les embeddings pour les chunks existants.

**Techniques clés** :
- Utilisation de SentenceTransformer pour la génération d'embeddings de haute qualité.
- Normalisation L2 pour améliorer la qualité des embeddings.
- Intégration avec ChromaDB pour un stockage et une recherche efficaces.

In [2]:
from sentence_transformers import SentenceTransformer
import chromadb
import os
import numpy as np
from typing import List, Dict
from sklearn.preprocessing import normalize

class EmbeddingGenerator:
    def __init__(self, model_name='BAAI/bge-large-en-v1.5'):
        self.model = SentenceTransformer(model_name)
        self.persist_directory = os.path.join(os.getcwd(), "chroma_db")
        self.chroma_client = chromadb.PersistentClient(path=self.persist_directory)
        self.collection = self.chroma_client.get_or_create_collection("document_embeddings")

    def generate_embeddings(self, chunks: List[Dict]):
        texts = [chunk.page_content for chunk in chunks]
        
        # Générer des embeddings avec SentenceTransformer
        embeddings = self.model.encode(texts, show_progress_bar=True)
        
        # Normalisation L2 des embeddings
        embeddings = normalize(embeddings)
        
        return embeddings

    def store_embeddings(self, chunks: List[Dict], embeddings: np.ndarray):
        ids = [str(i) for i in range(len(chunks))]
        metadatas = [
            {
                "source": chunk.metadata.get('source', ''),
                "title": chunk.metadata.get('title', ''),
                "article_num": chunk.metadata.get('article_num', '')
            } 
            for chunk in chunks
        ]
        texts = [chunk.page_content for chunk in chunks]
        
        self.collection.add(
            embeddings=embeddings.tolist(),
            documents=texts,
            metadatas=metadatas,
            ids=ids
        )

    def process_and_store(self, chunks: List[Dict]):
        embeddings = self.generate_embeddings(chunks)
        self.store_embeddings(chunks, embeddings)
        print(f"Generated and stored embeddings for {len(chunks)} chunks.")

    def get_collection_info(self):
        return {
            "name": self.collection.name,
            "count": self.collection.count()
        }

    def get_similar_chunks(self, query: str, n_results: int = 5):
        query_embedding = self.generate_embeddings([{"page_content": query}])
        results = self.collection.query(
            query_embeddings=query_embedding.tolist(),
            n_results=n_results,
            include=["documents", "metadatas", "distances"]
        )
        return results

    def update_embeddings(self, chunks: List[Dict]):
        """Mise à jour des embeddings pour les chunks existants."""
        embeddings = self.generate_embeddings(chunks)
        ids = [str(i) for i in range(len(chunks))]
        self.collection.update(
            ids=ids,
            embeddings=embeddings.tolist(),
            metadatas=[chunk.metadata for chunk in chunks],
            documents=[chunk.page_content for chunk in chunks]
        )
        print(f"Updated embeddings for {len(chunks)} chunks.")

c:\Users\Kaizen\Desktop\Test_peaks\chatbot_env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## 3. Recherche semantique à partir d'une entrée : `semantic_search.py`

### Classe `SemanticSearch`
- **Initialisation** :
  - Configure le modèle SentenceTransformer et la connexion à ChromaDB.
- **Méthode `search(query, n_results)`** :
  - Génère l'embedding de la requête.
  - Effectue une recherche dans ChromaDB.
  - Formate les résultats avec des scores de pertinence.
- **Méthode `get_relevant_context(query, max_length)`** :
  - Récupère et concatène les chunks pertinents jusqu'à une longueur maximale.

**Techniques clés** :
- Utilisation de la similitude cosinus pour le classement des résultats.
- Limitation de la longueur du contexte pour optimiser l'entrée du LLM.

In [3]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os

class SemanticSearch:
    def __init__(self, model_name='BAAI/bge-large-en-v1.5'):
        self.model = SentenceTransformer(model_name)
        self.persist_directory = os.path.join(os.getcwd(), "chroma_db")
        self.chroma_client = chromadb.PersistentClient(path=self.persist_directory)
        self.collection = self.chroma_client.get_collection("document_embeddings")

    def search(self, query, n_results=5):
        # Générer l'embedding pour la requête
        query_embedding = self.model.encode(query).tolist()

        # Effectuer la recherche dans ChromaDB
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results,
            include=["documents", "metadatas", "distances"]
        )

        # Formater les résultats
        formatted_results = []
        for doc, metadata, distance in zip(results['documents'][0], results['metadatas'][0], results['distances'][0]):
            formatted_results.append({
                'content': doc,
                'metadata': metadata,
                'relevance_score': 1 - distance  # Convertir la distance en score de pertinence
            })

        return formatted_results

    def get_relevant_context(self, query, max_length=1000):
        results = self.search(query)
        context = ""
        for result in results:
            if len(context) + len(result['content']) <= max_length:
                context += result['content'] + " "
            else:
                break
        return context.strip()


## 4. Interraction avec le modèle Mistral 7B(Ollama) : `llm_interface.py`

### Classe `LLMInterface`
- **Initialisation** : Configure l'URL de l'API locale (probablement Ollama).
- **Méthode `generate_response(prompt, max_tokens)`** :
  - Envoie une requête à l'API locale du LLM.
  - Gère les erreurs de connexion.
- **Méthode `simple_query(question)`** :
  - Formate une question simple pour le LLM.

**Techniques clés** :
- Utilisation d'une API locale pour l'inférence du LLM.
- Gestion des erreurs pour une meilleure robustesse.

In [4]:
import requests
import json

m1 = 'mistral'
m2 = 'phi3'

class LLMInterface:
    def __init__(self, model_name=m2):
        self.model_name = model_name
        self.api_url = "http://localhost:11434/api/generate"

    def generate_response(self, prompt, max_tokens=500):
        data = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False,
            "max_tokens": max_tokens
        }
        
        try:
            response = requests.post(self.api_url, json=data)
            response.raise_for_status()  # Raise an exception for bad status codes
            return json.loads(response.text)['response']
        except requests.exceptions.RequestException as e:
            return f"Error: Unable to generate response. {str(e)}"

    def simple_query(self, question):
        prompt = f"Human: {question}\n\nAssistant:"
        return self.generate_response(prompt)

## 5. Flux de travail (Main Script) : `main.py`

### Fonction `format_context(results)`
- Formate les résultats de la recherche pour le prompt du LLM.

### Fonction `main()`
- Initialise les composants (SemanticSearch, LLMInterface).
- Gère la boucle principale d'interaction avec l'utilisateur.
- Coordonne le processus de recherche, génération de réponse et affichage.

**Techniques clés** :
- Orchestration du flux de travail complet du chatbot.
- Formatage du prompt pour optimiser la réponse du LLM.
- Affichage des sources pour la transparence.

_Note : L'interface streamlit(`streamlit_app.py`) reproduit à peu près le même processus._

In [5]:
import os

def format_context(results):
    context = ""
    for i, result in enumerate(results[:3], 1):  # Utiliser les 3 premiers résultats
        context += f"Extrait {i}:\n"
        context += f"Titre: {result['metadata']['title']}\n"
        context += f"Article: {result['metadata']['article_num']}\n"
        context += f"Contenu: {result['content']}\n\n"
    return context.strip()

def main():
    pdf_path = "constitution.pdf"
    
    '''# Traitement du PDF (si ce n'est pas déjà fait)
    processor = PDFProcessor(pdf_path)
    chunks = processor.process()

    # Génération et stockage des embeddings (si ce n'est pas déjà fait)
    embedding_gen = EmbeddingGenerator()
    embedding_gen.process_and_store(chunks)'''

    # Création des objets de recherche sémantique et d'interface LLM
    searcher = SemanticSearch()
    llm = LLMInterface()

    print("Chatbot de la Constitution française")
    print("-----------------------------------")
    print("Posez vos questions sur la Constitution française. Tapez 'q' pour quitter.")

    while True:
        query = input("\nVotre question : ")
        if query.lower() == 'q':
            break

        # Recherche sémantique
        results = searcher.search(query, n_results=3)  # Récupérer les 3 meilleurs résultats

        # Formater le contexte pour le LLM
        context = format_context(results)

        # Préparer le prompt pour le LLM
        prompt = f"""En tant qu'expert en droit constitutionnel français, veuillez répondre à la question suivante en vous basant uniquement sur les extraits fournis de la Constitution française. Si les extraits ne contiennent pas suffisamment d'informations pour répondre à la question, indiquez-le clairement.

Extraits de la Constitution :
{context}

Question : {query}

Réponse :"""

        # Générer la réponse avec le LLM
        response = llm.generate_response(prompt)

        print("\nRéponse du chatbot :")
        print(response)

        # Afficher les sources utilisées
        print("\nSources utilisées :")
        for i, result in enumerate(results, 1):
            print(f"{i}. {result['metadata']['title']} - Article {result['metadata']['article_num']}")

if __name__ == "__main__":
    main()

Chatbot de la Constitution française
-----------------------------------
Posez vos questions sur la Constitution française. Tapez 'q' pour quitter.

Réponse du chatbot :
 Le Président de la République nomme le Premier ministre, comme indiqué dans l'extrait 1 de la Constitution française.

Sources utilisées :
1. Titre II 
LE PRÉSIDENT DE LA R ÉPUBLIQUE - Article 8
2. Titre III  
LE GOUVERNEMENT - Article 21
3. Titre II 
LE PRÉSIDENT DE LA R ÉPUBLIQUE - Article 13


# Pistes d'optimisations futures :
 
* Optimisation de la gestions des ressources pour accroître la vitesse de réponse du modèle.
* Optimisation des mécanismes de traîtement(chunks, embeddings) et des méthodes de calculs de similarités pour améliorer les performances lors des questions plus complexes.
* Améliorer les mécanismes de gestion contextuelle (Prompt engineering plus avancé).
* Intégration des mécanisme de gestion des exception (Par exemple, pour que le modèle réponde correctement lorsqu'il ne trouve pas l'information ou lorsqu'elle est absente).
* Amélioration de l'interface graphique.